# Script Demo

This is a really really basic demo of lecture script generation. The way it works is you edit the example.md found in the files tab and then run the following snippet

In [ ]:
from parse import parse_from_file
from upload import upload_script
import sys

filepath = "./example.md"

parsed = parse_from_file(filepath)
if parsed:
    response = upload_script(parsed)
    print(response)
else:
    print(parsed)